##### 시상파일을 읽어서 지점,사원,회사별로 작성 


In [8]:
import pandas as pd
import GladUtil as glad
import time
import os

COLS_NAME = ['지점','담당','보험회사','시상금','지사지급','구분']
C_DICT = {'한화':'한화손보','KB':'KB손보','농협':'농협손보','MG':'MG손보','롯데':'롯데손보','DB':'DB손보',
          '현대':'현대해상','메리츠':'메리츠화재','흥국':'흥국화재','AIG':'AIG손보','삼성':'삼성화재',
          '하나':'하나손보','처브':'처브손보'}
DIR_PATH = 'e:/글로벌백업(180329)/04.수수료작업/'

ym ='202312'
inf_nm = ym + '시상/'+ ym[0:4]+ ym[4:6]+ ' KJ시상정산.xlsx'
in_file = DIR_PATH + inf_nm                   #원장

data = pd.read_excel(in_file, sheet_name = None)

In [9]:
''' fc시상 '''
# 사원시상 정산 행
cols = data['정산'].iloc[86].values.tolist()
cols[:2] = ['지점','담당']
# 데이터부
dfw = data['정산'].iloc[88:].fillna(0)
dfw.columns = cols
dfw = dfw[dfw['계'] != 0 ].iloc[: ,:17].copy()
# 열을 행으로 변환
df_fc = (dfw.melt(id_vars=['지점','담당'])
            .rename(columns={'value':'시상금','variable':'보험회사'})
            .astype({'시상금':'float64'}))
# 보험사명 변환
df_fc['보험회사'] = df_fc['보험회사'].apply(lambda x: glad.convertByVal(x,C_DICT))
# fc시상 지급율 적용
dict_r = {'글래드시흥':94, '글래드조은':95, '글래드방이':95}
df_fc['지사지급'] = df_fc.apply(lambda x: x['시상금'] * dict_r.get(x['지점'],100) / 100, axis=1)
df_fc['구분' ] = 'FC_시상'

df_fc = df_fc[df_fc['보험회사'] != '계'][COLS_NAME]

In [4]:
''' 법인-생보 '''
df_life = data['생보'].groupby(['지점','담당','보험회사'])[['GLAD','지사시상합']].sum().reset_index()
df_life['구분'] = '법인_생보'
df_life.columns = COLS_NAME

'''법인-손보'''
df_long = data['손보'].groupby(['지점','담당','보험회사'])[['GLAD시상','지사시상']].sum().reset_index()
df_long['구분'] = '법인_장기'
df_long.columns = COLS_NAME

''' 법인-일반'''
df_gen = data['일반'].groupby(['지점','담당','보험회사'])[['시상금']].sum().reset_index()
df_gen['지사'] = 0
df_gen['구분'] = '법인_일반'
df_gen.columns = COLS_NAME

'''법인-자동차'''
df_car = data['자동차'].groupby(['지점','담당','보험회사'])[['시상금']].sum().reset_index()
df_car['지사'] = 0
df_car['구분'] = '법인_자동차'
df_car.columns = COLS_NAME

In [6]:
'''2차년도 시상'''
inf_nm = '시상이자/'+ '본사시상정산_2차년도_'+ym[0:4]+ ym[4:6]+ '.xlsx'
in_file = DIR_PATH + inf_nm                   #원장
data = pd.read_excel(in_file, sheet_name = None)

dfw = pd.DataFrame()
for k in data.keys():
    if k !='요약':
        df1 = data[k][['보험사','지점','사원','정산금']]
        dfw = pd.concat([dfw, df1])

df_year2 = dfw.groupby(['지점','사원','보험사'])[['정산금']].sum().reset_index()
df_year2['지급'] = 0
df_year2['구분'] = '본사_2차년도'
df_year2.columns = COLS_NAME

''' 모든 시상을 합침 '''
df_all = pd.concat([df_fc, df_life, df_long, df_car, df_gen, df_year2])
df_all = df_all[df_all['시상금']!=0]

In [7]:
df_all.to_clipboard(index=False)